In [1]:
%matplotlib inline
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import os
import itertools
from tqdm import *
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import math
import matplotlib.pyplot as plt

from IPython.display import clear_output

from io import StringIO
import re

import pickle
import os

from multiprocessing import Pool 
from itertools import product
import sys
from functools import partial

# Step 1: Load Data Set

In [2]:
fileNames = []
for file in os.listdir("data/train_1"):
    if file.endswith(".jpg"):
        fileNames.append(file)
# fileNamesDF = pd.DataFrame(fileNames)

In [16]:
#only take certain artist so that the probabilities aren't basically all false when doing the tupples
minNumPaintingsPerArtist = 10

# #for each artist, only take n of their paintings
numPaintingsPerAuthor = 100


trainInfo = pd.read_csv('data/train_info.csv')
trainInfo = trainInfo[trainInfo['filename'].isin(fileNames)]

mostPopularArtists = trainInfo['artist'].value_counts()[trainInfo['artist'].value_counts() > minNumPaintingsPerArtist]
# print(mostPopularArtists.head())
# fileNamesDFAll = trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)]
# print(fileNamesDFAll.head())



fileNamesDF = pd.DataFrame()
for artist in tqdm(mostPopularArtists.index):
    fileNamesDF = fileNamesDF.append(trainInfo[trainInfo['artist'] == artist][:numPaintingsPerAuthor])

In [18]:
len(fileNamesDF)

11025

In [5]:
# Load the all_data_info for the dataset we are using
allInfo = pd.read_csv('data/all_data_info.csv')
allInfo = allInfo[allInfo['new_filename'].isin(fileNamesDF['filename'])]

featuresDF = pd.DataFrame(allInfo['new_filename'])
featuresDF['pixelsx'] = allInfo['pixelsx']
featuresDF['pixelsy'] = allInfo['pixelsy']
# featuresDF['features'] = pd.Series()

In [6]:
def normalizeSeries(series, maxNum):
    series = series/maxNum
    return series

In [7]:
def featurizeImage(filename):
    # Extract the features from the actual image
    jpgfile = Image.open("data/train_1/" + filename)
#     print(jpgfile)
    
    bands = jpgfile.getbands()
    if bands[0] == 'L':
        # grayscale image
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['grayscale'])
#         imageDF['grayscale'] = normalizeSeries(imageDF['grayscale',])
    elif len(bands) == 4:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue','alpha'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
        imageDF['alpha'] = normalizeSeries(imageDF['alpha'],255)  
    elif len(bands) == 3:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
    else:
        imageDF = pd.DataFrame(list(jpgfile.getdata()))
        print(bands)
        return imageDF.mean(), bands

    return imageDF.mean()

In [8]:
def pickleDF(fileName,dataFrame,numSplits):
    print(numSplits)
    dfArray = np.array_split(dataFrame, numSplits)
    for i, df in enumerate(dfArray):
        df.to_pickle(fileName+"-"+str(i))

In [9]:
def getImgData(dataFrame,baseName):
    sys.stdout.flush()
#     index = 0
#     while os.path.exists(baseName+"-"+str(index)):
#         index = index + 1
    
    newFileName = baseName + ".pckl"
    print(newFileName)
    sys.stdout.flush()
    featureSeries = []
    for row in tqdm(list(dataFrame.index), leave=False):        
        imageFeatures = featurizeImage(dataFrame.loc[row]['new_filename'])
        featureSeries.append(imageFeatures)
        sys.stdout.flush()
    
#     print(featureSeries)
    dataFrame['features'] = featureSeries
#     print(dataFrame['features'])
    dataFrame.to_pickle(newFileName)
    return dataFrame

In [10]:
def concatFrames(dataFrames):
    return pd.concat(dataFrames)

In [11]:
def loadImagesFromPickles(fileNames):
    
    dataFrames = []
    for fileName in fileNames:
        dataFrames.append(pd.read_pickle(fileName))

    return concatFrames(dataFrames)

In [12]:
baseFileName = 'trainImgsAllcolorBands'
splitFeaturesData = np.array_split(featuresDF,10)
print(len(splitFeaturesData))
pool = Pool()
inputData = [[splitFeaturesData[i], baseFileName+"-"+str(i)] for i in range(len(splitFeaturesData))]
pool.starmap(getImgData, inputData)
pool.close() 
results = pool.join()

10
trainImgsAllcolorBands-0.pckl
trainImgsAllcolorBands-1.pckl
trainImgsAllcolorBands-2.pckl
trainImgsAllcolorBands-3.pckl


trainImgsAllcolorBands-4.pckl


trainImgsAllcolorBands-5.pckl


trainImgsAllcolorBands-6.pckl


trainImgsAllcolorBands-7.pckl


trainImgsAllcolorBands-8.pckl


trainImgsAllcolorBands-9.pckl


In [13]:
print(results)

None


In [14]:
pickleLoaded = loadImagesFromPickles([baseFileName+"-"+str(i)+".pckl" for i in range(len(splitFeaturesData))])

In [15]:
pickleLoaded

,new_filename,pixelsx,pixelsy,features
1870,15075.jpg,2536.0,2929.0,red 0.884975 green 0.884975 blue 0...
1921,18039.jpg,2352.0,3129.0,red 0.456600 green 0.302188 blue 0...
2026,1053.jpg,2536.0,2836.0,red 0.834556 green 0.834556 blue 0...
2032,1458.jpg,2394.0,3000.0,red 0.83587 green 0.83587 blue 0.8...
2067,18076.jpg,2232.0,3198.0,red 0.542533 green 0.392234 blue 0...
2110,13849.jpg,2360.0,3000.0,red 0.779067 green 0.779067 blue 0...
2452,1805.jpg,2220.0,3000.0,red 0.457848 green 0.457848 blue 0...
2496,19349.jpg,2536.0,2602.0,red 0.558449 green 0.491345 blue 0...
2522,10199.jpg,2187.0,3000.0,red 0.629660 green 0.572854 blue 0...
2540,12105.jpg,2179.0,3000.0,red 0.797854 green 0.797854 blue 0...
